In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("gtfintechlab/FOMC-RoBERTa")

model = AutoModelForSequenceClassification.from_pretrained("gtfintechlab/FOMC-RoBERTa")

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

In [6]:
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

NameError: name 'emotions' is not defined

In [10]:
num_labels = 6

In [11]:
model = (AutoModelForSequenceClassification
        .from_pretrained(model_ckpt, num_labels=num_labels)\
        .to(device))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.we

In [12]:
from sklearn.metrics import accuracy_score, f1_score

In [13]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1}

In [14]:
from huggingface_hub import notebook_login

In [15]:
# notebook_login()

In [16]:
from transformers import Trainer, TrainingArguments

In [17]:
emotions_encoded.set_format('torch',
                           columns=['input_ids', 'attention_mask', 'label'])

In [18]:
batch_size = 64
logging_steps = len(emotions_encoded['train']) // batch_size
model_name = f'{model_ckpt}-finetuned-emotion'
training_args = TrainingArguments(output_dir=model_name,
                                 num_train_epochs=2,
                                 learning_rate=2e-5,
                                 per_device_train_batch_size=batch_size,
                                 per_gpu_eval_batch_size=batch_size,
                                 weight_decay=0.01,
                                 evaluation_strategy='epoch',
                                 disable_tqdm=False,
                                 logging_steps=logging_steps,
                                 push_to_hub=False,
                                 log_level='error'
                                 )

In [19]:
len(emotions_encoded['train'])

16000

In [20]:
from transformers import Trainer

In [21]:
trainer = Trainer(model=model,
                  args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=emotions_encoded['train'],
                  eval_dataset=emotions_encoded['validation'],
                  tokenizer=tokenizer)

In [22]:
trainer.train() 

C:\Users\ftrav\anaconda3\envs\pytorchenv\lib\site-packages\transformers\optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.823800,0.322954,0.903000,0.900274
2,0.248000,0.216912,0.927000,0.927135


TrainOutput(global_step=500, training_loss=0.5358845138549805, metrics={'train_runtime': 1142.8519, 'train_samples_per_second': 28.0, 'train_steps_per_second': 0.438, 'total_flos': 720342861696000.0, 'train_loss': 0.5358845138549805, 'epoch': 2.0})

In [23]:
preds_output = trainer.predict(emotions_encoded['validation'])

In [24]:
preds_output.metrics

{'test_loss': 0.21691159904003143,
 'test_accuracy': 0.927,
 'test_f1': 0.9271345963162837,
 'test_runtime': 8.167,
 'test_samples_per_second': 244.888,
 'test_steps_per_second': 3.918}

In [25]:
y_preds = np.argmax(preds_output.predictions, axis=1)

In [26]:
y_preds

array([0, 0, 2, ..., 1, 1, 1], dtype=int64)

In [27]:
compute_metrics(preds_output)

{'accuracy': 0.927, 'f1': 0.9271345963162837}

In [28]:
# import matplotlib.pyplot as plt
# from sklearn.metrics import plot_confusion_matrix

# plot_confusion_matrix(y_preds)

In [29]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    # Place all input tensors on the same device as the model
    inputs = {k:v.to(device) for k,v in batch.items() 
              if k in tokenizer.model_input_names}

    with torch.no_grad():
        output = model(**inputs)
        pred_label = torch.argmax(output.logits, axis=-1)
        loss = cross_entropy(output.logits, batch["label"].to(device), 
                             reduction="none")

    # Place outputs on CPU for compatibility with other dataset columns   
    return {"loss": loss.cpu().numpy(), 
            "predicted_label": pred_label.cpu().numpy()}
     

In [30]:
# Convert our dataset back to PyTorch tensors
emotions_encoded.set_format("torch", 
                            columns=["input_ids", "attention_mask", "label"])
# Compute loss values
emotions_encoded["validation"] = emotions_encoded["validation"].map(
    forward_pass_with_label, batched=True, batch_size=16)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [31]:
emotions_encoded.set_format("pandas")
cols = ["text", "label", "predicted_label", "loss"]
df_test = emotions_encoded["validation"][:][cols]
# df_test["label"] = df_test["label"].apply(label_int2str)
# df_test["predicted_label"] = (df_test["predicted_label"].apply(label_int2str))

In [32]:
df_test.sort_values("loss", ascending=False).head(10)

,text,label,predicted_label,loss
1500,i guess we would naturally feel a sense of lon...,3,0,5.528459
1963,i called myself pro life and voted for perry w...,1,0,5.122855
1950,i as representative of everything thats wrong ...,5,0,5.116637
1274,i am going to several holiday parties and i ca...,1,0,5.072585
1870,i guess i feel betrayed because i admired him ...,1,0,5.001413
882,i feel badly about reneging on my commitment t...,2,0,4.909636
465,i would eventually go in to these stores but i...,1,4,4.877432
765,i feel super awkward and out of place right now,1,0,4.874084
1801,i feel that he was being overshadowed by the s...,2,0,4.854888
1111,im lazy my characters fall into categories of ...,1,4,4.411380


In [33]:
from transformers import pipeline

# # Change `transformersbook` to your Hub username
# model_id = "transformersbook/distilbert-base-uncased-finetuned-emotion"
classifier = pipeline("text-classification", model=model)
tokenizer=

Exception: Impossible to guess which tokenizer to use. Please provide a PreTrainedTokenizer class or a path/identifier to a pretrained tokenizer.

In [ ]:
custom_tweet = "I saw a movie today and it was really good."
preds = classifier(custom_tweet, top_k=None)

In [ ]:
preds_df = pd.DataFrame(preds)
preds_df['label']

In [ ]:
plt.bar(preds_df['label'], 100 * preds_df["score"], color='C0')
plt.title(f'"{custom_tweet}"')
plt.ylabel("Class probability (%)")
plt.show()